In [2]:
import numpy as np
import itertools

In [3]:
c = 3

In [4]:
# Generate some data
np.random.seed(42)
lambda1 = np.random.normal(size=(c, c))
lambda2 = np.random.normal(size=(c, c))
lambda3 = np.random.normal(size=(c, c))
G1 = np.random.normal(size=(c, c, c))
G2 = np.random.normal(size=(c, c, c))
U = np.random.normal(size=(c, c, c, c))

In [5]:
def Z_naive(lambda1, lambda2, lambda3, G1, G2, U):
    c = lambda1.shape[0]
    Z = np.zeros(shape=(c, c, c, c))
    for a, b, c, d, e, f, g, h, i, j in itertools.product(*([range(c)]*10)):
        Z[a, h, i, j] += lambda1[a, b]*lambda2[d, e]*lambda3[g, h]*G1[c, b, d]*G2[f, e, g]*U[i, j, c, f]
    return Z

In [6]:
%%timeit
Z = Z_naive(lambda1, lambda2, lambda3, G1, G2, U)

61 ms ± 353 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
Z = Z_naive(lambda1, lambda2, lambda3, G1, G2, U)
Z.shape

(3, 3, 3, 3)

# Homework

Using kernel: Python 3.11.5 locally

Let's take a look at `einsum_path` output for our convolution:

In [8]:
path, repr = np.einsum_path("ab,de,gh,cbd,feg,ijcf", lambda1, lambda2, lambda3, G1, G2, U, optimize="optimal")
print(f"{path}\n\n{repr}")

['einsum_path', (0, 3), (0, 2), (0, 3), (1, 2), (0, 1)]

  Complete contraction:  ab,de,gh,cbd,feg,ijcf->ahij
         Naive scaling:  10
     Optimized scaling:  6
      Naive FLOP count:  3.543e+05
  Optimized FLOP count:  2.431e+03
   Theoretical speedup:  145.740
  Largest intermediate:  8.100e+01 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   4                 cbd,ab->acd                 de,gh,feg,ijcf,acd->ahij
   4                 feg,de->dfg                    gh,ijcf,acd,dfg->ahij
   4                 dfg,gh->dfh                       ijcf,acd,dfh->ahij
   5               dfh,acd->acfh                          ijcf,acfh->ahij
   6             acfh,ijcf->ahij                               ahij->ahij


According to the result, the minimal number of flops is $\chi^6$ as opposed to $\chi^{10}$ of the naive implementation.
The function suggests the following sequence of convolutions:

$Z_{ahij} = \sum_{bcdefg}\lambda^{(1)}{}_{ab}\Gamma^{(1)}{}_{cbd}\lambda^{(2)}{}_{de}\Gamma^{(2)}{}_{feg}\lambda^{(3)}{}_{gh}U_{ijcf} = $

$= \sum_{cdefg} T^{(1)}{}_{acd} \lambda^{(2)}{}_{de}\Gamma^{(2)}{}_{feg}\lambda^{(3)}{}_{gh}U_{ijcf} = 
\sum_{cdfg} T^{(1)}{}_{acd} T^{(2)}{}_{dfg} \lambda^{(3)}{}_{gh}U_{ijcf} =$

$= \sum_{cdf} T^{(1)}{}_{acd} T^{(3)}{}_{dfh} U_{ijcf} = \sum_{cf} T^{(4)}{}_{acfh} U_{ijcf}$.


Let's write a function that mimicks the behaviour of `numpy.einsum` and calculates convolutions in the same fashion using only the primitive `numpy.tensordot`:

In [9]:
def Z_optimal(lambda1, lambda2, lambda3, G1, G2, U):
    args = [lambda1, lambda2, lambda3, G1, G2, U]
    contraction_list = [((3, 0), {'b'}, 'cbd','ab', 'acd', [2, 0, 1]),
                        ((2, 0), {'e'}, 'feg', 'de', 'dfg', [2, 0, 1]),
                        ((3, 0), {'g'}, 'dfg', 'gh', 'dfh', [0, 1, 2]),
                        ((2, 1), {'d'}, 'dfh', 'acd', 'acfh', [2, 3, 0, 1]),
                        ((1, 0), {'c', 'f'}, 'acfh', 'ijcf', 'ahij', [0, 1, 2, 3])]
    
    for inds, idx_rm, input_left, input_right, results_index, permutations in contraction_list:
        # inds - a tuple of two: list indices of the tensors that are contracted in the current iteration
        # idx_rm - tensor indices that are being contracred
        # input_left, input_right - tensor indices of the tensors being contracted
        # results_index - tensor index of the resulting tensor
        # permutations - list of permutations (as supplied to numpy.transpose) needed to
        #   get desired index since after tensordot() the index of the resulting tensor
        #   may not be the same as result_index.
        arg_left = args.pop(inds[0])
        arg_right = args.pop(inds[1])
        tensor_result = (input_left + input_right)
        for s in idx_rm:
            tensor_result.replace(s, "")
        left_pos, right_pos = [], []
        for s in sorted(idx_rm):
            left_pos.append(input_left.find(s))
            right_pos.append(input_right.find(s))
        new_view = np.tensordot(arg_left, arg_right, axes=(tuple(left_pos), tuple(right_pos)))
        if results_index != tensor_result:
            new_view = np.transpose(new_view, permutations)
        args.append(new_view)
    return args[0]

Let's check the implementation agains `Z_naive` and `numpy.einsum`:

In [10]:
Z = Z_naive(lambda1, lambda2, lambda3, G1, G2, U)
Q = Z_optimal(lambda1, lambda2, lambda3, G1, G2, U)
O = np.einsum("ab,de,gh,cbd,feg,ijcf", lambda1, lambda2, lambda3, G1, G2, U, optimize="optimal")
print(np.linalg.norm(Z - Q))
print(np.linalg.norm(O - Q))

3.803298653358641e-13
0.0


Finally, let's time the function:

In [11]:
%%timeit

Q = Z_optimal(lambda1, lambda2, lambda3, G1, G2, U)

65.8 µs ± 562 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%%timeit

O = np.einsum("ab,de,gh,cbd,feg,ijcf", lambda1, lambda2, lambda3, G1, G2, U, optimize="optimal")

903 µs ± 6.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
